# Función para prueba

In [71]:
import pandas as pd
import numpy as np
import requests

API_URL = "http://localhost:8000/predict"

def llamar_api_predict(csv_path=None, n_samples=10):


    if csv_path is None:
        raise ValueError("Debes proporcionar la ruta al CSV.")

    # Cargar dataset
    df = pd.read_csv(csv_path)
    var_objective = ["is_canceled"]

    # Crear columnas derivadas
    df["arrival_date"] = pd.to_datetime(
        df["arrival_date_year"].astype(str) + "-" +
        df["arrival_date_month"].astype(str) + "-" +
        df["arrival_date_day_of_month"].astype(str)
    )
    df["day_of_week"] = df["arrival_date"].dt.dayofweek.astype(int)
    df["is_weekend"] = df["day_of_week"].isin([5, 6]).astype(int)

    # Tomar muestra
    df = df.dropna().sample(n=n_samples)

    # Guardar valores originales
    y_true = df[var_objective].reset_index(drop=True)

    # Quitar columna objetivo antes de enviar
    df = df.drop(var_objective, axis=1)

    # Preparar payload (convertir tipos a JSON-friendly)
    df = df.applymap(lambda x: x.isoformat() if isinstance(x, pd.Timestamp) else x)
    df = df.where(pd.notnull(df), None)
    df = df.apply(lambda s: s.map(lambda v: v.item() if isinstance(v, np.generic) else v))

    payload = {"data": df.to_dict(orient="records")}

    try:
        r = requests.post(API_URL, json=payload, timeout=30)
        r.raise_for_status()
        resp = r.json()
    except requests.exceptions.RequestException as e:
        print("Error al llamar a la API:", e)
        return None

    if "predictions" in resp:
        preds = pd.DataFrame({"prediction": resp["predictions"]})
        comparacion = pd.concat([y_true.reset_index(drop=True), preds], axis=1)
        return comparacion
    else:
        print("\nRespuesta sin clave 'predictions':")
        print(resp)
        return None


#Pruebas 

In [83]:
# Utilizando dataset hotel_bookings.csv


llamar_api_predict(csv_path="data/hotel_bookings.csv", n_samples=6)




C:\Users\brand\AppData\Local\Temp\ipykernel_23280\3308821770.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.isoformat() if isinstance(x, pd.Timestamp) else x)


,is_canceled,prediction
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
5,0,0
